### Import Libraries

In [1]:
import arcpy
import pandas as pd
import numpy as np
import os
import censusdata as cs
from census import Census

pd.set_option('display.max_rows',None, 'display.max_columns',None,'display.max_colwidth', None)

## Set Directory

In [2]:
path = 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness'
os.chdir(path)

In [3]:
ProjectGDB =  r"C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\"
arcpy.env.overwriteOutput = True
arcpy.env.workspace = ProjectGDB

# Market Demand

- Business Analyst

In [4]:
arcpy.ba.EnrichLayer(
    "ArlingtonMarketAssessmentAreasPT",
    "ArlingtonMarketAssessmentAreasEnrich",
    ############ Population Totals ################
    "populationtotals.TOTPOP_CY;populationtotals.POPDENS_CY;populationtotals.TOTPOP_FY;HistoricalHousing.TOTHU_CY;" \
    #######  Vacancy #####
    "vacant.VACANT_CY;vacant.VACANT_FY;housingunittotals.TOTHU_CY;homevalue.MEDVAL_CY;homevalue.ACSMEDVAL;" \
    ################# Retail ##################################
    "RetailDemandbyNAICS.INDRTFD_X;RetailDemandbyNAICS.INDRT_X;RetailDemandbyNAICS.IND722_X;" \
    ###################  Employment ###############################
    "EmploymentUnemployment.UNEMPRT_CY;KeyUSFacts.MEDVAL_CY;KeyUSFacts.MEDVAL_FY;" \
    ################################## Crime#####################
    "crime.CRMCYTOTC;businesses.N01_BUS;housingcosts.MEDCRNT_CY",
    "STRAIGHT_LINE",
    1,
    "Miles"
)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketAssessmentAreasEnrich'>

In [6]:
print(arcpy.Describe("ArlingtonMarketAssessmentAreas").spatialReference.name)

NAD_1983_StatePlane_Texas_North_Central_FIPS_4202_Feet


In [7]:
print(arcpy.management.CheckGeometry("ArlingtonMarketAssessmentAreas", "check_geom"))

C:\Users\Thepr\Documents\Consulting\Market Readiness\Market Readiness.gdb\check_geom


In [8]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable('ArlingtonMarketAssessmentAreasEnrich')

AssessmentAreas = arcpy.management.AddJoin("ArlingtonMarket","Ranking","ArlingtonMarketAssessmentAreasEnrich",
                         "Ranking","KEEP_ALL","NO_INDEX_JOIN_FIELDS","NO_REBUILD_INDEX")

arcpy.conversion.ExportFeatures(AssessmentAreas,"ArlingtonMarketAssessmentAreas","","NOT_USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketAssessmentAreas'>

- Residential Permits

In [10]:
ResidentialPermits = arcpy.management.SelectLayerByAttribute(
    "Issued_Permits",
    "NEW_SELECTION",
    """ SUBDESC IN ('Single-Family','Two-Family','Residential','Residential Pool Barrier','Residential Privacy',
        'Residential Spa','Residential Swimming Pool','Garage Conversion','Detached Garage',
        'Permanent Storage/Accy Building (over 320 sq ft)','Portable Storage/Accy Building (320 sq ft and under)',
        'Patio Cover/Gazebo','Carport','Retaining Wall' )
        
    AND WORKDESC IN ('Addition','Demolition', 'Foundation','Interior Finish','Move','New','New Construction',
    'New Construction - Shell','Remodel/Alterations/Repairs','Remove/Replace','Repair', 'Repair / Replacement','Reroof',
    'Solar Panel', 'Window/Door Replacement')
    """)

ResidentialPermcount = int(arcpy.management.GetCount(ResidentialPermits)[0])
print(ResidentialPermcount)

10463


In [11]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(ResidentialPermits)

analysisfields = ['FOLDERYEAR', 'STATUSDESC', 'ISSUEDATE', 'FINALDATE', 'SUBDESC', 'FOLDERNAME',
                  'Constructi','MainUse','LandUseDes','Structure','Census','NameofBusi','SignConstr',
                  'ServiceGri','FOLDERCOND','ImportDate']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('ResidentialPermits'):
    arcpy.Delete_management('ResidentialPermits')

arcpy.conversion.ExportFeatures(ResidentialPermits,"ResidentialPermits","","NOT_USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ResidentialPermits'>

- Near Analysis Residential Development

In [12]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketAssessmentAreas")
fieldmapping.addTable("ResidentialPermits")

analysisfields = ["ImportDate","FOLDERYEAR","STATUSDESC","ISSUEDATE","FINALDATE","SUBDESC",
                  "FOLDERNAME", "Constructi","MainUse","LandUseDes","Structure","Census","NameofBusi",
                  "SignConstr","ServiceGri","FOLDERCOND"]

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketAssessmentAreas",
                           "ResidentialPermits","ArlingtonResNearby",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonResNearby","Join_Count","NearbyResidentialDevelopment","NearbyResidentialDevelopment")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonResNearby'>

- Commercial Permits

In [13]:
CommercialPermits = arcpy.management.SelectLayerByAttribute(
 "Issued_Permits",
    "NEW_SELECTION",
    """
    SUBDESC IN ('Assembly','Awning', 'Billboard', 'Boat Dock', 'Building Identification Sign', 'Business',
    'Canopy / Fascia Sign', 'Clean and Show', 'Commercial Privacy', 'Commercial Screening', 'Decks',
    'Directional',  'Directory',  'Educational',  'Entryway Sign',  'Existing Business/New Owner',
    'Expanding Lease Space',  'Factory',  'Ground',  'Institutional', 'Manufactured Housing',
    'Marquee Sign','Menu Board', 'Mercantile',  'Multi Tenant Ground Sign', 'New Tenant',
    'Onsite Development', 'Other Undefined Sign', 'Pole Sign','Projecting (Blade) Sign',
    'Same Business Owner/New Business Name','Shell Building/House Lights', 'Storage',
    'Subdivision Screening Wall', 'Temporary Construction Office', 'Temporary Sales Office',
    'Utility & Miscellaneous', 'Wall Sign'
    )
    AND WORKDESC IN ( 'New','New Construction','New Construction - Shell','Interior Finish',
    'Remodel/Alterations/Repairs', 'Repair', 'Repair / Replacement','Remove/Replace', 'Reroof',
    'Face Change', 'Foundation', 'Solar Panel', 'Window/Door Replacement','Demolition' )
    """
)

CommercialPermcount = int(arcpy.management.GetCount(CommercialPermits)[0])
print(CommercialPermcount)

4595


In [14]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(CommercialPermits)

analysisfields = ['FOLDERYEAR', 'STATUSDESC', 'ISSUEDATE', 'FINALDATE', 'SUBDESC', 'FOLDERNAME',
                  'Constructi','MainUse','LandUseDes','Structure','Census','NameofBusi','SignConstr',
                  'ServiceGri','FOLDERCOND','ImportDate']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('CommercialPermits'):
    arcpy.Delete_management('CommercialPermits')

arcpy.conversion.ExportFeatures(CommercialPermits,"CommercialPermits","","NOT_USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\CommercialPermits'>

- Near Analysis Commercial Development

In [15]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonResNearby")
fieldmapping.addTable("CommercialPermits")

analysisfields = ["ImportDate","FOLDERYEAR","STATUSDESC","ISSUEDATE","FINALDATE","SUBDESC",
                  "FOLDERNAME", "Constructi","MainUse","LandUseDes","Structure","Census","NameofBusi",
                  "SignConstr","ServiceGri","FOLDERCOND"]

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonResNearby",
                           "CommercialPermits","ArlingtonComNearby",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonComNearby","Join_Count","NearbyCommercialDevelopment","NearbyCommercialDevelopment")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonComNearby'>

In [16]:
NewConstruction = arcpy.management.SelectLayerByAttribute(
 "Issued_Permits",
    "NEW_SELECTION",
    """
    WORKDESC IN ('New Construction - Shell', 'New Construction')
    """
)

NewConstructionCount = int(arcpy.management.GetCount(NewConstruction)[0])
print(NewConstructionCount)

3243


In [17]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(NewConstruction)

analysisfields = ['FOLDERYEAR', 'STATUSDESC', 'ISSUEDATE', 'FINALDATE', 'SUBDESC', 'FOLDERNAME',
                  'Constructi','MainUse','LandUseDes','Structure','Census','NameofBusi','SignConstr',
                  'ServiceGri','FOLDERCOND','ImportDate']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('NewConstruction'):
    arcpy.Delete_management('NewConstruction')

arcpy.conversion.ExportFeatures(NewConstruction,"NewConstruction","","NOT_USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\NewConstruction'>

- Near Analysis construction Activity

In [18]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonComNearby")
fieldmapping.addTable("NewConstruction")

analysisfields = ["ImportDate","FOLDERYEAR","STATUSDESC","ISSUEDATE","FINALDATE","SUBDESC",
                  "FOLDERNAME", "Constructi","MainUse","LandUseDes","Structure","Census","NameofBusi",
                  "SignConstr","ServiceGri","FOLDERCOND"]

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonComNearby",
                           "NewConstruction","ArlingtonMarketReadiness",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadiness","Join_Count","NewConstructionActivity","NewConstructionActivity")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadiness'>

# Proximity to Amenity/Access

### Proximity to Schools
- Spatial join schools within 1 mile buffer to selected areas and count the number of schools

In [19]:
cols = []
for row in arcpy.ListFields('SchoolsPointsArlington'):
    cols.append(row.name)
cols  

['OBJECTID',
 'Shape',
 'DISTNAME',
 'CAMPNAME',
 'STREET',
 'CITY',
 'ZIP',
 'INSTR_TYPE',
 'XCOORD',
 'YCOORD',
 'DOWNLOADDATE',
 'GRADEGROUP',
 'State']

In [20]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadiness")
fieldmapping.addTable("SchoolsPointsArlington")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadiness",
                           "SchoolsPointsArlington","ArlingtonMarketReadinessSchools",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessSchools","Join_Count","NearbySchools","NearbySchools")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessSchools'>

### Proximity to Hospitals
- Spatial join hospitals within 1 mile buffer to selected areas and count the number of hospitals

In [21]:
cols = []
for row in arcpy.ListFields('Healthcare_Facilities'):
    cols.append(row.name)
cols  

['OBJECTID_1',
 'Shape',
 'OBJECTID',
 'BUILDINGCO',
 'ADDRESS',
 'HOSPITALNA',
 'Type']

In [22]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessSchools")
fieldmapping.addTable("Healthcare_Facilities")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessSchools",
                           "Healthcare_Facilities","ArlingtonMarketReadinessHealthcare",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessHealthcare","Join_Count","Nearbyhealthcare","Nearbyhealthcare")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessHealthcare'>

### Proximity to Grocery

In [23]:
cols = []
for row in arcpy.ListFields('Grocery'):
    cols.append(row.name)
cols  

['OBJECTID_1',
 'Shape',
 'ObjectId',
 'Store_Name',
 'Address',
 'City',
 'State',
 'County',
 'Longitude',
 'Latitude',
 'StoreType',
 'Produce']

In [24]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessHealthcare")
fieldmapping.addTable("Grocery")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessHealthcare",
                           "Grocery","ArlingtonMarketReadinessGrocery",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessGrocery","Join_Count","NearbyGrocery","NearbyGrocery")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessGrocery'>

### Proximity to Shopping Centers, Restaurants and Entertainment 
- This indicator only have data for Food estabishments and Downtown restuarants

In [25]:
cols = []
for row in arcpy.ListFields('DowntownRestuarants'):
    cols.append(row.name)
cols  

['OBJECTID',
 'Shape',
 'Type',
 'Tags',
 'ServesAlcohol',
 'Name',
 'Address',
 'City',
 'State',
 'ZIP',
 'Phone',
 'OnlineReviewLink',
 'NavigateToLink',
 'CreationDate',
 'Creator',
 'EditDate',
 'Editor']

In [26]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessGrocery")
fieldmapping.addTable("FoodEstablishmentDowntownRestaurants")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce',
                 'OBJECTID_1','Shape','ImportDate','OBJECTID','FacilityNa','Establishm','PropertyAd',
                  'Inspection','Inspecti_1','Attempt','XCoord','YCoord','Inspecti_2', 'Type','Tags','ServesAlcohol',
                  'Name','Address','City','State','ZIP','Phone','OnlineReviewLink','NavigateToLink','CreationDate',
                  'Creator','EditDate','Editor']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessGrocery",
                           "FoodEstablishmentDowntownRestaurants","ArlingtonMarketReadinessFoodEstablishments",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessFoodEstablishments","Join_Count","NearbyRestuarants",
                            "NearbyRestuarants")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessFoodEstablishments'>

### Proximity to Parks

- Converted the Parks Polygon to point and counted the number of parks that are in proximity to the selected areas

In [27]:
arcpy.management.FeatureToPoint("Park2024","ArlingtonParks2024","CENTROID")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonParks2024'>

In [28]:
cols = []
for row in arcpy.ListFields('ArlingtonParks2024'):
    cols.append(row.name)
cols  

['OBJECTID_1',
 'Shape',
 'OBJECTID',
 'NAME',
 'OWNER',
 'NOTES',
 'CITY',
 'FEATTYPE',
 'STATUS',
 'SOURCE',
 'SOURCE_YEA',
 'ShapeSTLen',
 'ShapeSTAre',
 'ORIG_FID']

In [29]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessFoodEstablishments")
fieldmapping.addTable("ArlingtonParks2024")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce',
                 'OBJECTID_1','Shape','ImportDate','OBJECTID','FacilityNa','Establishm','PropertyAd',
                  'Inspection','Inspecti_1','Attempt','XCoord','YCoord','Inspecti_2', 'Type','Tags','ServesAlcohol',
                  'Name','Address','City','State','ZIP','Phone','OnlineReviewLink','NavigateToLink','CreationDate',
                  'Creator','EditDate','Editor','OBJECTID_1','Shape','OBJECTID','NAME','OWNER','NOTES','CITY',
                  'FEATTYPE','STATUS','SOURCE', 'SOURCE_YEA', 'ShapeSTLen', 'ShapeSTAre', 'ORIG_FID']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessFoodEstablishments",
                           "ArlingtonParks2024","ArlingtonMarketReadinessParks",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessParks","Join_Count","NearbyParks","NearbyParks")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessParks'>

- Walkability

In [30]:
# Intersect Selected Areas and EPA NWI

arcpy.analysis.Intersect("ArlingtonMarketReadinessParks #;NationalWalkabilityIndex #",
                         "ArlingtonMarketReadiness_Intersect","ALL",None,"INPUT")


# Add Field to calculated weighted area

arcpy.management.AddField("ArlingtonMarketReadiness_Intersect","WeightedWalk","DOUBLE",None,None,None,"WeightedWalk",
                          "NULLABLE","NON_REQUIRED","")

# Compute the Weighted Walk

arcpy.management.CalculateField("ArlingtonMarketReadiness_Intersect","WeightedWalk","!Acres! * !NatWalkInd!",
                                "PYTHON3","","DOUBLE","NO_ENFORCE_DOMAINS")


# Create summary statistics, the Ranking is the Id

arcpy.analysis.Statistics("ArlingtonMarketReadiness_Intersect","WalkabilityStatistics",
                          "WeightedWalk SUM;Acres SUM","Ranking","")

# Join summary to Market Readiness data

walkability = arcpy.management.AddJoin("ArlingtonMarketReadinessParks","Ranking","WalkabilityStatistics",
                         "Ranking","KEEP_ALL","NO_INDEX_JOIN_FIELDS","NO_REBUILD_INDEX")

arcpy.conversion.ExportFeatures(walkability,"ArlingtonMarketReadinessEPAWI",
                                "","NOT_USE_ALIAS",'',None)

# Add Field to calculated weighted area

arcpy.management.AddField("ArlingtonMarketReadinessEPAWI","WalkabilityIndex","DOUBLE",None,None,None,"WalkabilityIndex",
                          "NULLABLE","NON_REQUIRED","")

# Compute the Weighted Walk

arcpy.management.CalculateField("ArlingtonMarketReadinessEPAWI","WalkabilityIndex","!SUM_WeightedWalk! / !SUM_Acres!",
                                "PYTHON3","","DOUBLE","NO_ENFORCE_DOMAINS")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessEPAWI'>

In [32]:
cols = []
for row in arcpy.ListFields('ArlingtonMarketReadinessEPAWI'):
    cols.append(row.aliasName)
cols  

['OBJECTID',
 'Shape',
 'NearbyParks',
 'TARGET_FID',
 'NearbyRestuarants',
 'NearbyGrocery',
 'Nearbyhealthcare',
 'NearbySchools',
 'NewConstructionActivity',
 'NearbyCommercialDevelopment',
 'NearbyResidentialDevelopment',
 'TARGET_FID',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Ranking',
 'Location',
 'Value',
 'Override',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Ranking',
 'Location',
 'Value',
 'Override',
 'aggregationMethod',
 'HasData',
 'ORIGINAL_OID',
 'sourceCountry',
 'apportionmentConfidence',
 'populationToPolygonSizeRating',
 'areaType',
 'bufferUnits',
 'bufferUnit',
 'bufferRadii',
 '2025 Total Population',
 '2025 Population Density',
 '2030 Total Population',
 '2025 Total Housing Units',
 '2025 Vacant Housing Units',
 '2030 Vacant Housing Units',
 '2025 Total Housing Units',
 '2025 Median Home Value',
 '2023 Median Home Value (ACS 5-Yr)',
 '2025 NAICS 44-45,722 Expenditures',
 '2025 NAICS 44-45 Expenditures',
 '2025 NAICS 722 Expenditures',

- Distance to Interstate/Major Highways

In [33]:
arcpy.analysis.Near("ArlingtonMarketReadinessEPAWI","TxDOT_Roadway",None,"NO_LOCATION","NO_ANGLE","PLANAR",
                    "NEAR_DIST NEAR_DIST","Miles")

arcpy.management.AlterField("ArlingtonMarketReadinessEPAWI","NEAR_DIST","DistanceToHighway","DistanceToHighway")


<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessEPAWI'>

- Distance to Regional Rail

In [34]:
arcpy.analysis.Near("ArlingtonMarketReadinessEPAWI","Texas_Railroads",None,"NO_LOCATION","NO_ANGLE","PLANAR",
                    "NEAR_DIST NEAR_DIST","Miles")

arcpy.management.AlterField("ArlingtonMarketReadinessEPAWI","NEAR_DIST","DistanceToRegionalRail","DistanceToRegionalRail")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessEPAWI'>

- Distance to DFW/Love Field/Grand Praire

In [35]:
arcpy.analysis.Near("ArlingtonMarketReadinessEPAWI","Airport_Boundary",None,"NO_LOCATION","NO_ANGLE","PLANAR",
                    "NEAR_DIST NEAR_DIST","Miles")

arcpy.management.AlterField("ArlingtonMarketReadinessEPAWI","NEAR_DIST","DistanceToAirport","DistanceToAirport")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessEPAWI'>

# Infrastructure Capacity

#### Fire and police station
- merged the fire and police station points, then computed the count of each stations within the selected areas


In [36]:
cols = []
for row in arcpy.ListFields('FireandPoliceStations'):
    cols.append(row.name)
cols  

['OBJECTID_1',
 'Shape',
 'OBJECTID',
 'STATIONNAM',
 'PARCELADDR',
 'STRUCTUREA',
 'DISTRICTID',
 'STATION']

In [37]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessEPAWI")
fieldmapping.addTable("FireandPoliceStations")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce',
                 'OBJECTID_1','Shape','ImportDate','OBJECTID','FacilityNa','Establishm','PropertyAd',
                  'Inspection','Inspecti_1','Attempt','XCoord','YCoord','Inspecti_2', 'Type','Tags','ServesAlcohol',
                  'Name','Address','City','State','ZIP','Phone','OnlineReviewLink','NavigateToLink','CreationDate',
                  'Creator','EditDate','Editor','OBJECTID_1','Shape','OBJECTID','NAME','OWNER','NOTES','CITY',
                  'FEATTYPE','STATUS','SOURCE', 'SOURCE_YEA', 'ShapeSTLen', 'ShapeSTAre', 'ORIG_FID',
                 'OBJECTID_1','Shape','OBJECTID','STATIONNAM','PARCELADDR','STRUCTUREA','DISTRICTID','STATION',
                 'Walkscore','OBJECTID','Ranking_1','FREQUENCY','SUM_WeightedWalk','SUM_Acres','Shape_Length','Shape_Area',
                 'NEAR_FID']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessEPAWI",
                           "FireandPoliceStations","ArlingtonMarketReadinessPoliceFireStations",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessPoliceFireStations","Join_Count","NearbyPolice","NearbyPolice")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessPoliceFireStations'>

In [39]:
cols = []
for row in arcpy.ListFields('ArlingtonMarketReadinessPoliceFireStations'):
    cols.append(row.aliasName)
cols  

['OBJECTID',
 'Shape',
 'NearbyPolice',
 'TARGET_FID',
 'NearbyParks',
 'NearbyRestuarants',
 'NearbyGrocery',
 'Nearbyhealthcare',
 'NearbySchools',
 'NewConstructionActivity',
 'NearbyCommercialDevelopment',
 'NearbyResidentialDevelopment',
 'TARGET_FID',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Ranking',
 'Location',
 'Value',
 'Override',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Location',
 'Value',
 'Override',
 'aggregationMethod',
 'HasData',
 'ORIGINAL_OID',
 'sourceCountry',
 'apportionmentConfidence',
 'populationToPolygonSizeRating',
 'areaType',
 'bufferUnits',
 'bufferUnit',
 'bufferRadii',
 '2025 Total Population',
 '2025 Population Density',
 '2030 Total Population',
 '2025 Total Housing Units',
 '2025 Vacant Housing Units',
 '2030 Vacant Housing Units',
 '2025 Total Housing Units',
 '2025 Median Home Value',
 '2023 Median Home Value (ACS 5-Yr)',
 '2025 NAICS 44-45,722 Expenditures',
 '2025 NAICS 44-45 Expenditures',
 '2025 NAICS 722 Expenditu

- Traffic Count

In [40]:
cols = []
for row in arcpy.ListFields('TrafficCountyHistoric5Year'):
    cols.append(row.name)
cols  

['OBJECTID',
 'Shape',
 'DIST_NM',
 'CNTY_NM',
 'TRFC_STATN',
 'LATEST_AAD',
 'AADT_RPT_Q',
 'AADT_RPT_H',
 'AADT_RPT_1',
 'AADT_RPT_2',
 'AADT_RPT_3']

In [41]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessPoliceFireStations")
fieldmapping.addTable("TrafficCountyHistoric5Year")

index = fieldmapping.findFieldMapIndex('AADT_RPT_Q')
fmap = fieldmapping.getFieldMap(index)
fmap.mergeRule = 'Mean'
fieldmapping.replaceFieldMap(index, fmap)


analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce',
                 'OBJECTID_1','Shape','ImportDate','OBJECTID','FacilityNa','Establishm','PropertyAd',
                  'Inspection','Inspecti_1','Attempt','XCoord','YCoord','Inspecti_2', 'Type','Tags','ServesAlcohol',
                  'Name','Address','City','State','ZIP','Phone','OnlineReviewLink','NavigateToLink','CreationDate',
                  'Creator','EditDate','Editor','OBJECTID_1','Shape','OBJECTID','NAME','OWNER','NOTES','CITY',
                  'FEATTYPE','STATUS','SOURCE', 'SOURCE_YEA', 'ShapeSTLen', 'ShapeSTAre', 'ORIG_FID',
                 'OBJECTID_1','Shape','OBJECTID','STATIONNAM','PARCELADDR','STRUCTUREA','DISTRICTID','STATION',
                 'Walkscore','OBJECTID','Ranking_1','FREQUENCY','SUM_WeightedWalk','SUM_Acres','Shape_Length','Shape_Area',
                 'OBJECTID','Shape','DIST_NM','CNTY_NM','TRFC_STATN','LATEST_AAD', 'AADT_RPT_H','TARGET_FID_12',
                  'AADT_RPT_1', 'AADT_RPT_2','AADT_RPT_3','Join_Count','TARGET_FID']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessPoliceFireStations",
                           "TrafficCountyHistoric5Year","ArlingtonMarketReadinessTraffic",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessTraffic","AADT_RPT_Q","Avg5yrTraffic","Avg5yrTraffic")
arcpy.management.AlterField("ArlingtonMarketReadinessTraffic","Join_Count","NearbyTrafficSensors","NearbyTrafficSensors")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessTraffic'>

In [42]:
cols = []
for row in arcpy.ListFields('ArlingtonMarketReadinessTraffic'):
    cols.append(row.aliasName)
cols 

['OBJECTID',
 'Shape',
 'NearbyTrafficSensors',
 'TARGET_FID',
 'NearbyPolice',
 'NearbyParks',
 'NearbyRestuarants',
 'NearbyGrocery',
 'Nearbyhealthcare',
 'NearbySchools',
 'NewConstructionActivity',
 'NearbyCommercialDevelopment',
 'NearbyResidentialDevelopment',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Ranking',
 'Location',
 'Value',
 'Override',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Location',
 'Value',
 'Override',
 'aggregationMethod',
 'HasData',
 'ORIGINAL_OID',
 'sourceCountry',
 'apportionmentConfidence',
 'populationToPolygonSizeRating',
 'areaType',
 'bufferUnits',
 'bufferUnit',
 'bufferRadii',
 '2025 Total Population',
 '2025 Population Density',
 '2030 Total Population',
 '2025 Total Housing Units',
 '2025 Vacant Housing Units',
 '2030 Vacant Housing Units',
 '2025 Total Housing Units',
 '2025 Median Home Value',
 '2023 Median Home Value (ACS 5-Yr)',
 '2025 NAICS 44-45,722 Expenditures',
 '2025 NAICS 44-45 Expenditures',
 '2025 NAICS 722

- **Environmental and Protected Areas**

In [43]:
cols = []
for row in arcpy.ListFields('Brownfields'):
    cols.append(row.aliasName)
cols 

['OBJECTID_1',
 'Shape',
 'OBJECTID',
 'TaxParcelI',
 'TaxParcelA',
 'TaxLegalKe',
 'MoreInform',
 'ShapeSTAre',
 'ShapeSTLen',
 'Shape_Length',
 'Shape_Area']

In [44]:
arcpy.management.FeatureToPoint("Brownfields","Brownfields2024","CENTROID")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\Brownfields2024'>

In [45]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessTraffic")
fieldmapping.addTable("Brownfields")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce',
                 'OBJECTID_1','Shape','ImportDate','OBJECTID','FacilityNa','Establishm','PropertyAd',
                  'Inspection','Inspecti_1','Attempt','XCoord','YCoord','Inspecti_2', 'Type','Tags','ServesAlcohol',
                  'Name','Address','City','State','ZIP','Phone','OnlineReviewLink','NavigateToLink','CreationDate',
                  'Creator','EditDate','Editor','OBJECTID_1','Shape','OBJECTID','NAME','OWNER','NOTES','CITY',
                  'FEATTYPE','STATUS','SOURCE', 'SOURCE_YEA', 'ShapeSTLen', 'ShapeSTAre', 'ORIG_FID',
                 'OBJECTID_1','Shape','OBJECTID','STATIONNAM','PARCELADDR','STRUCTUREA','DISTRICTID','STATION',
                 'Walkscore','OBJECTID','Ranking_1','FREQUENCY','SUM_WeightedWalk','SUM_Acres','Shape_Length','Shape_Area',
                 'OBJECTID_1', 'Shape','OBJECTID', 'TaxParcelI', 'TaxParcelA', 'TaxLegalKe', 'MoreInform', 'ShapeSTAre',
                  'ShapeSTLen']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessTraffic",
                           "Brownfields","ArlingtonMarketReadinessBrownfields",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessBrownfields","Join_Count","NearbyBrownfields","NearbyBrownfields")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessBrownfields'>

- **Road Conditions**

In [46]:
## Add field to road segment 

arcpy.management.AddField("StreetCondition2024","segmentlength","DOUBLE",None,None,None,"segmentlength",
                          "NULLABLE","NON_REQUIRED","")


arcpy.management.CalculateGeometryAttributes(
    in_features="StreetCondition2024",
    geometry_property=[["segmentlength", "LENGTH_GEODESIC"]],
    length_unit="MILES_US",
    coordinate_system=arcpy.SpatialReference(32038)  # NAD 1983 StatePlane Texas North Central Feet
)

# Add fields for Weighted length

arcpy.management.AddField("StreetCondition2024", "OCIxLen", "DOUBLE")

# Calculate field 

arcpy.management.CalculateField("StreetCondition2024", "OCIxLen", "!EstimatedOCI! * !segmentlength!", "PYTHON3")

#  Spatially join street condtion layer to last brownfields layer
arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessBrownfields","StreetCondition2024",
                           "ArlingtonMarketReadinessStreetcondt","JOIN_ONE_TO_MANY",
                           "KEEP_COMMON",'',"INTERSECT","1 Miles","",None)

# Grab summary statistics 

arcpy.analysis.Statistics("ArlingtonMarketReadinessStreetcondt","ArlingtonMarketReadinessStreetConditionSummary",
                          [["OCIxLen", "SUM"], ["segmentlength", "SUM"]],
                          "Ranking"  )

# Add field to for weighted OCI

arcpy.management.AddField("ArlingtonMarketReadinessStreetConditionSummary", "WeightedOCI", "DOUBLE")

# Calculate field

arcpy.management.CalculateField("ArlingtonMarketReadinessStreetConditionSummary","WeightedOCI",
                                "!SUM_OCIxLen! / !SUM_segmentlength!","PYTHON3")

# Add street condition 

streetcondition = arcpy.management.AddJoin("ArlingtonMarketReadinessBrownfields","Ranking",
                                           "ArlingtonMarketReadinessStreetConditionSummary","Ranking",
                                           "KEEP_ALL","NO_INDEX_JOIN_FIELDS","NO_REBUILD_INDEX")

# export final out 

fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(streetcondition)

analysisfields = ['SUM_OCIxLen','SUM_segmentlength','FREQUENCY']

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
arcpy.conversion.ExportFeatures(streetcondition,"ArlingtonMarketReadinessStreetCondition",
                                "","NOT_USE_ALIAS",fieldmapping,None)

# Alterfield name 
arcpy.management.AlterField("ArlingtonMarketReadinessStreetCondition","WeightedOCI",
                            "StreetCondition","StreetCondition")

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

...

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessStreetCondition'>

- **Zoning**

In [47]:
zoningfields = ['Community_Commercial','Entertainment_District_Overlay___Community_Commercial',
                'General_Commercial','Lamar_Collins_Mixed_Use_Overlay___Community_Commercial',
                'Lamar_Collins_Mixed_Use_Overlay___General_Commercial',
                'Lamar_Collins_Mixed_Use_Overlay___Planned_Development',
                'Neighborhood_Commercial','Office_Commercial',
                'Planned_Development___Residential_Medium_Density___Community_Com',
                'Planned_Development___Residential_Multi_Family_22___Community_Co',
                'Residential_Medium_Density','Residential_Multi_Family_22',
                'Village_on_the_Green']


In [48]:
desc = arcpy.Describe("ZoningDesignation")
print(desc.shapeType)

Polygon


In [50]:
cols = []
for row in arcpy.ListFields('ArlingtonMarketReadinessStreetCondition'):
    cols.append(row.aliasName)
cols

['OBJECTID',
 'Shape',
 'NearbyBrownfields',
 'TARGET_FID',
 'NearbyTrafficSensors',
 'NearbyPolice',
 'NearbyParks',
 'NearbyRestuarants',
 'NearbyGrocery',
 'Nearbyhealthcare',
 'NearbySchools',
 'NewConstructionActivity',
 'NearbyCommercialDevelopment',
 'NearbyResidentialDevelopment',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Ranking',
 'Location',
 'Value',
 'Override',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Location',
 'Value',
 'Override',
 'aggregationMethod',
 'HasData',
 'ORIGINAL_OID',
 'sourceCountry',
 'apportionmentConfidence',
 'populationToPolygonSizeRating',
 'areaType',
 'bufferUnits',
 'bufferUnit',
 'bufferRadii',
 '2025 Total Population',
 '2025 Population Density',
 '2030 Total Population',
 '2025 Total Housing Units',
 '2025 Vacant Housing Units',
 '2030 Vacant Housing Units',
 '2025 Total Housing Units',
 '2025 Median Home Value',
 '2023 Median Home Value (ACS 5-Yr)',
 '2025 NAICS 44-45,722 Expenditures',
 '2025 NAICS 44-45 Expenditu

In [51]:
arcpy.analysis.Intersect(['ZoningDistrict', 'ArlingtonMarketReadinessStreetCondition'],
                         'ZoningDesignation',"ALL", None, "INPUT")


arcpy.management.AddField('ZoningDesignation', "Acreage", "DOUBLE")

arcpy.management.CalculateGeometryAttributes("ZoningDesignation",[["Acreage", "AREA"]], "", "ACRES_US",
                                             coordinate_system=arcpy.SpatialReference(32038),
                                             coordinate_format="SAME_AS_INPUT")

# Summarize acreage by Ranking + Zoning
arcpy.analysis.Statistics( 'ZoningDesignation', 'ZoningSummary', [["Acreage", "SUM"]], ["Ranking", "Zoning"])

# Pivot: one column per zoning type
arcpy.management.PivotTable('ZoningSummary', "Ranking", "Zoning", "SUM_Acreage", 'ZoningPivot')

# Prevent duplicate fields

delete = [f for f in zoningfields if f in
          [field.name for field in arcpy.ListFields('ArlingtonMarketReadinessStreetCondition')]]

if delete:
    arcpy.management.DeleteField('ArlingtonMarketReadinessStreetCondition',";".join(delete),"DELETE_FIELDS")

# Join back
pivot = arcpy.management.JoinField("ArlingtonMarketReadinessStreetCondition","Ranking","ZoningPivot","Ranking")

# Export final
arcpy.conversion.ExportFeatures( pivot, "ArlingtonMarketReadinessZoning")


existingfields = [f.name for f in arcpy.ListFields("ArlingtonMarketReadinessZoning")]

for field in zoningfields:
    if field not in existingfields:
        arcpy.management.AddField("ArlingtonMarketReadinessZoning", field, "DOUBLE")
    
    expr = f"(!{field}! / float(!Acres!)) if !Acres! not in (None, 0) else 0"
    arcpy.management.CalculateField("ArlingtonMarketReadinessZoning", field, expr, "PYTHON3")

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

...

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [52]:
existingfields = [f.name for f in arcpy.ListFields("ArlingtonMarketReadinessZoning")]

if "TotalZoningAcres" not in existingfields:
    arcpy.management.AddField("ArlingtonMarketReadinessZoning","TotalZoningAcres", "DOUBLE")

# Wrap each field so None is treated as 0
sumexpr = " + ".join([f"(0 if !{f}! is None else !{f}!)" for f in zoningfields])

# Calculate the total
arcpy.management.CalculateField("ArlingtonMarketReadinessZoning", "TotalZoningAcres", sumexpr, "PYTHON3")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessZoning'>

### Export Data as CSV

In [5]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessZoning")

analysisfields = ['OBJECTID','Shape','Shape_Length','Shape_Area','Ranking_1', 'Value','Descriptio', 'TARGET_FID', 
                  'Override','aggregationMethod','HasData', 'ORIGINAL_OID','sourceCountry',
                  'apportionmentConfidence','Community_Commercial',
                  'Entertainment_District_Overlay___Community_Commercial','General_Commercial',
                  'Lamar_Collins_Mixed_Use_Overlay___Community_Commercial',
                  'Lamar_Collins_Mixed_Use_Overlay___General_Commercial',
                  'Lamar_Collins_Mixed_Use_Overlay___Planned_Development', 'Neighborhood_Commercial','Office_Commercial',
                  'Planned_Development___Residential_Medium_Density___Community_Com',
                  'Planned_Development___Residential_Multi_Family_22___Community_Co','Residential_Medium_Density',
                  'Residential_Multi_Family_22', 'Village_on_the_Green','NearbyTrafficSensors','Shape_Leng', 
                  'Comments', 'Acres','populationToPolygonSizeRating']

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.conversion.ExportTable("ArlingtonMarketReadinessZoning","ArlingtonMarketReadiness.csv",
                             'Ranking IN (1,2,3,4,5,6,7,8)',"USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\ArlingtonMarketReadiness.csv'>

In [6]:
Data = pd.read_csv('ArlingtonMarketReadiness.csv')
Data

,NearbyBrownfields,NearbyPolice,NearbyParks,NearbyRestuarants,NearbyGrocery,Nearbyhealthcare,NearbySchools,NewConstructionActivity,NearbyCommercialDevelopment,NearbyResidentialDevelopment,Ranking,Location,Shape_Leng,Comments,Acres,Descriptio,Location_1,Value,Override,areaType,bufferUnits,bufferUnit,bufferRadii,2025_Total_Population,2025_Population_Density,2030_Total_Population,2025_Total_Housing_Units,2025_Vacant_Housing_Units,2030_Vacant_Housing_Units,2025_Total_Housing_Units_1,2025_Median_Home_Value,2023_Median_Home_Value__ACS_5_Yr_,2025_NAICS_44_45_722_Expenditures,2025_NAICS_44_45_Expenditures,2025_NAICS_722_Expenditures,2025_Unemployment_Rate,2025_Median_Home_Value_1,2030_Median_Home_Value,2024_Total_Crime_Index,2025_Total_Businesses__NAICS_,2025_Median_Monthly_Contract_Rent,Ranking_1,WalkabilityIndex,DistanceToHighway,DistanceToRegionalRail,DistanceToAirport,Avg5yrTraffic,StreetCondition,Ranking_12,TotalZoningAcres
0,109,2,4,408,15,1,8,80,176,394,1.0,E Park Row Dr and New York Ave (Shortlist FBC area),7247.264003,,50.891592,E Park Row Dr and New York Ave (Shortlist FBC area),NaN,4.428571,4.428571,RingBuffer,esriMiles,Miles,1.0,26516.0,8447.0,25893.0,8833.0,466.0,585.0,NaN,196409.0,189692.0,139745867.0,117668034.0,22077833.0,4.1,NaN,265301.0,104.0,396.0,1267.0,NaN,7.966667,0.404905,1.124177,1.911533,6671,60.462982,NaN,0.882877
1,95,1,6,424,13,4,4,61,451,337,6.0,S Cooper St and W Pioneer Pkwy (Shortlist FBC area),5911.284414,,48.888828,S Cooper St and W Pioneer Pkwy (Shortlist FBC area),NaN,3.428571,3.428571,RingBuffer,esriMiles,Miles,1.0,18305.0,5831.3,18273.0,8208.0,849.0,887.0,NaN,270710.0,231597.0,161989548.0,136940185.0,25049363.0,5.0,NaN,320594.0,159.0,1118.0,1366.0,NaN,6.952381,0.000000,1.885454,2.376831,10543,55.963405,NaN,0.661347
2,8,1,3,26,0,2,4,182,35,465,8.0,W Harris Rd and US 287 FWY,9116.152470,,49.269876,W Harris Rd and US 287 FWY,NaN,3.142857,3.142857,RingBuffer,esriMiles,Miles,1.0,6268.0,1996.8,6682.0,2425.0,71.0,53.0,NaN,438569.0,386743.0,84890096.0,71845725.0,13044371.0,3.5,NaN,496051.0,106.0,125.0,1430.0,NaN,7.083333,0.024719,1.147230,4.606631,3641,64.136194,NaN,0.830904
3,97,1,5,639,7,3,4,133,402,430,4.0,Lamar and Collins St,6566.982036,,45.439520,Lamar and Collins St,NaN,3.714286,3.714286,RingBuffer,esriMiles,Miles,1.0,18204.0,5799.1,18722.0,10510.0,905.0,936.0,NaN,362186.0,243822.0,204352523.0,172565979.0,31786544.0,4.5,NaN,418875.0,228.0,845.0,1315.0,NaN,7.333333,0.001256,1.531286,4.861033,6340,62.576728,NaN,0.743863
4,65,1,3,188,11,0,4,44,192,140,5.0,Gateway/Boundary: SH 360 FWY and E Pioneer Pkwy,11171.806092,,51.884581,Gateway/Boundary: SH 360 FWY and E Pioneer Pkwy,NaN,3.428571,3.430000,RingBuffer,esriMiles,Miles,1.0,16369.0,5214.6,16417.0,6569.0,478.0,479.0,NaN,233705.0,205134.0,98758054.0,83160340.0,15597714.0,4.0,NaN,293407.0,127.0,593.0,1248.0,NaN,7.416667,0.000000,0.348636,0.288725,4845,61.243132,NaN,0.000644
5,175,1,2,309,1,1,0,31,437,32,2.0,Gateway: E Road to Six Flags and Six Flags Dr,6774.866358,,53.108891,Gateway: E Road to Six Flags and Six Flags Dr,NaN,4.285714,4.285714,RingBuffer,esriMiles,Miles,1.0,1730.0,551.1,1671.0,856.0,46.0,66.0,NaN,445833.0,293750.0,13749163.0,11588100.0,2161063.0,0.6,NaN,466667.0,258.0,845.0,1325.0,NaN,12.166667,0.014302,0.243234,3.204295,4738,64.743129,NaN,0.569670
6,52,0,4,486,6,7,2,49,580,255,3.0,S Cooper St at Parks Mall,6851.550711,,47.898735,S Cooper St at Parks Mall,NaN,3.857143,3.857143,RingBuffer,esriMiles,Miles,1.0,9784.0,3116.8,9683.0,4169.0,234.0,310.0,NaN,309720.0,264340.0,90907355.0,76779993.0,14127362.0,2.2,NaN,345896.0,199.0,1070.0,1456.0,NaN,11.333333,0.031055,3.616936,1.656529,10681,56.907856,NaN,0.951380
7,10,1,5,103,2,0,7,36,149,411,7.0,Dr. MLK Dr. adj. to Airport,9253.733242,,51.487463,Dr. MLK Dr. adj. to Airport,NaN,3.142857,3.142857,RingBuffer,esriMiles,Miles,1.0,12877.0,4102.1,12503.0,4128.0,74.0,147.0,NaN,285602.0,244726.0,110761056.0,93670628.0,17090428.0,3.8,NaN,342595.0,76.0,186.0,1736.0,NaN,9.166667,0.994194,4.625314,0.000000,56

In [7]:
Data.columns

Index(['NearbyBrownfields', 'NearbyPolice', 'NearbyParks', 'NearbyRestuarants',
       'NearbyGrocery', 'Nearbyhealthcare', 'NearbySchools',
       'NewConstructionActivity', 'NearbyCommercialDevelopment',
       'NearbyResidentialDevelopment', 'Ranking', 'Location', 'Shape_Leng',
       'Comments', 'Acres', 'Descriptio', 'Location_1', 'Value', 'Override',
       'areaType', 'bufferUnits', 'bufferUnit', 'bufferRadii',
       '2025_Total_Population', '2025_Population_Density',
       '2030_Total_Population', '2025_Total_Housing_Units',
       '2025_Vacant_Housing_Units', '2030_Vacant_Housing_Units',
       '2025_Total_Housing_Units_1', '2025_Median_Home_Value',
       '2023_Median_Home_Value__ACS_5_Yr_',
       '2025_NAICS_44_45_722_Expenditures', '2025_NAICS_44_45_Expenditures',
       '2025_NAICS_722_Expenditures', '2025_Unemployment_Rate',
       '2025_Median_Home_Value_1', '2030_Median_Home_Value',
       '2024_Total_Crime_Index', '2025_Total_Businesses__NAICS_',
       '2025_Median

## Scoring 

In [8]:
Data = pd.read_csv('ArlingtonMarketReadiness.csv')
Data.columns

Index(['NearbyBrownfields', 'NearbyPolice', 'NearbyParks', 'NearbyRestuarants',
       'NearbyGrocery', 'Nearbyhealthcare', 'NearbySchools',
       'NewConstructionActivity', 'NearbyCommercialDevelopment',
       'NearbyResidentialDevelopment', 'Ranking', 'Location', 'Shape_Leng',
       'Comments', 'Acres', 'Descriptio', 'Location_1', 'Value', 'Override',
       'areaType', 'bufferUnits', 'bufferUnit', 'bufferRadii',
       '2025_Total_Population', '2025_Population_Density',
       '2030_Total_Population', '2025_Total_Housing_Units',
       '2025_Vacant_Housing_Units', '2030_Vacant_Housing_Units',
       '2025_Total_Housing_Units_1', '2025_Median_Home_Value',
       '2023_Median_Home_Value__ACS_5_Yr_',
       '2025_NAICS_44_45_722_Expenditures', '2025_NAICS_44_45_Expenditures',
       '2025_NAICS_722_Expenditures', '2025_Unemployment_Rate',
       '2025_Median_Home_Value_1', '2030_Median_Home_Value',
       '2024_Total_Crime_Index', '2025_Total_Businesses__NAICS_',
       '2025_Median

### Criteria for group

In [9]:
criteria = [
    # Access group (10 criteria) - each gets 1/10 = 0.10
    ("NearbySchools", "Access", 0.10, "benefit"),
    ("NearbyRestuarants", "Access", 0.10, "benefit"),
    ("NearbyGrocery", "Access", 0.10, "benefit"),
    ("WalkabilityIndex", "Access", 0.10, "benefit"),
    ("NearbyParks", "Access", 0.10, "benefit"),
    ("Nearbyhealthcare", "Access", 0.10, "benefit"),
    ("DistanceToHighway", "Access", 0.10, "cost"),  
    ("DistanceToRegionalRail", "Access", 0.10, "cost"), 
    ("DistanceToAirport", "Access", 0.10, "cost"),  
    ("2024 Total Crime Index", "Access", 0.10, "cost"),
    
    # Market group (10 criteria) - each gets 1/10 = 0.10
    ("NewConstructionActivity", "Market", 0.10, "benefit"),
    ("NearbyCommercialDevelopment", "Market", 0.10, "benefit"),
    ("2025_NAICS_44_45_722_Expenditures", "Market", 0.10, "benefit"),
    ("NearbyResidentialDevelopment", "Market", 0.10, "benefit"),
    ("2025_Unemployment_Rate", "Market", 0.10, "cost"),
    ("2030_Total_Population", "Market", 0.10, "benefit"),
    ("2025_Median_Home_Value", "Market", 0.10, "benefit"),
    ("2025 Total Businesses (NAICS)", "Market", 0.10, "benefit"),
    ("2025 Vacant Housing Units", "Market", 0.10, "cost"),  # Note: changed to cost
    ("2025 Median Monthly Contract Rent", "Market", 0.10, "benefit"),
    
    # Infra group (3 criteria) - each gets 1/3 = 0.333
    ("StreetCondition", "Infra", 0.333, "benefit"),
    ("Avg5yrTraffic", "Infra", 0.333, "benefit"),
    ("NearbyPolice", "Infra", 0.334, "benefit"),  # Slightly higher to sum to 1.0
    
    # Reg group (2 criteria) - each gets 1/2 = 0.50
    ("TotalZoningAcres", "Reg", 0.50, "benefit"),
    ("NearbyBrownfields", "Reg", 0.50, "cost"),
]

## Map fields to data

In [10]:
# Manual mapping for known mismatches
manual_mapping = {
    "2024 Total Crime Index": "2024_Total_Crime_Index",
    "2025 Total Businesses (NAICS)": "2025_Total_Businesses__NAICS_",
    "2025 Vacant Housing Units": "2025_Vacant_Housing_Units", 
    "2025 Median Monthly Contract Rent": "2025_Median_Monthly_Contract_Rent"
}

field_mapping = {}
missing_fields = []

for fld, group, weight, direction in criteria:
    # Check manual mapping first
    if fld in manual_mapping and manual_mapping[fld] in Data.columns:
        field_mapping[fld] = manual_mapping[fld]
        print(f"Manual mapped '{fld}' -> '{manual_mapping[fld]}'")
    # Try exact match
    elif fld in Data.columns:
        field_mapping[fld] = fld
    else:
        # Try partial matching for truncated names
        matched = False
        for col in Data.columns:
            # More flexible matching
            if (col.replace("_", " ").replace("__", " ").lower().startswith(fld[:15].lower()) or 
                fld.replace(" ", "_").lower().startswith(col[:15].lower()) or
                col.replace("_", "").replace(" ", "").lower() in fld.replace(" ", "").replace("(", "").replace(")", "").lower()):
                field_mapping[fld] = col
                matched = True
                print(f"Pattern mapped '{fld}' -> '{col}'")
                break
        
        if not matched:
            missing_fields.append(fld)
            print(f"Warning: Could not find column for '{fld}'")

if missing_fields:
    print(f"\nMissing fields: {missing_fields}")
    # Filter out missing criteria
    criteria = [(fld, group, weight, direction) for fld, group, weight, direction in criteria 
               if fld not in missing_fields]
    print(f"Proceeding with {len(criteria)} criteria")

Manual mapped '2024 Total Crime Index' -> '2024_Total_Crime_Index'
Manual mapped '2025 Total Businesses (NAICS)' -> '2025_Total_Businesses__NAICS_'
Manual mapped '2025 Vacant Housing Units' -> '2025_Vacant_Housing_Units'
Manual mapped '2025 Median Monthly Contract Rent' -> '2025_Median_Monthly_Contract_Rent'


## Calculate percentiles 

In [11]:
# --- 3. Calculate percentiles for standardization ---
print("\nCalculating percentiles for standardization...")
stats = {}

for fld, group, weight, direction in criteria:
    actual_field = field_mapping[fld]
    clean_vals = Data[actual_field].dropna()
    
    if len(clean_vals) > 0:
        p5, p95 = np.percentile(clean_vals, [5, 95])
        stats[fld] = (p5, p95)
        print(f"{actual_field}: P5={p5:.2f}, P95={p95:.2f}, n={len(clean_vals)}")
    else:
        stats[fld] = (0, 1)
        print(f"Warning: No valid values for {actual_field}")



Calculating percentiles for standardization...
NearbySchools: P5=0.70, P95=7.65, n=8
NearbyRestuarants: P5=52.95, P95=585.45, n=8
NearbyGrocery: P5=0.35, P95=14.30, n=8
WalkabilityIndex: P5=7.00, P95=11.88, n=8
NearbyParks: P5=2.35, P95=5.65, n=8
Nearbyhealthcare: P5=0.00, P95=5.95, n=8
DistanceToHighway: P5=0.00, P95=0.79, n=8
DistanceToRegionalRail: P5=0.28, P95=4.27, n=8
DistanceToAirport: P5=0.10, P95=4.77, n=8
2024_Total_Crime_Index: P5=85.80, P95=247.50, n=8
NewConstructionActivity: P5=32.75, P95=164.85, n=8
NearbyCommercialDevelopment: P5=74.90, P95=534.85, n=8
2025_NAICS_44_45_722_Expenditures: P5=38648489.55, P95=189525481.75, n=8
NearbyResidentialDevelopment: P5=69.80, P95=452.75, n=8
2025_Unemployment_Rate: P5=1.16, P95=4.82, n=8
2030_Total_Population: P5=3424.85, P95=23383.15, n=8
2025_Median_Home_Value: P5=209462.60, P95=443290.60, n=8
2025_Total_Businesses__NAICS_: P5=146.35, P95=1101.20, n=8
2025_Vacant_Housing_Units: P5=54.75, P95=885.40, n=8
2025_Median_Monthly_Contra

### Standardization 

In [12]:
for fld, group, weight, direction in criteria:
    actual_field = field_mapping[fld]
    p5, p95 = stats[fld]
    
    # Create standardized column name
    std_col_name = f"STD_{fld.replace(' ', '_').replace('(', '').replace(')', '')[:15]}"
    
    # Get the raw values
    raw_values = Data[actual_field]
    
    # Standardize based on direction
    if p95 == p5:
        # Avoid division by zero
        Data[std_col_name] = 0.5
    else:
        # Clip to percentile range
        clipped_values = raw_values.clip(lower=p5, upper=p95)
        
        if direction == "benefit":
            # Higher values are better
            standardized = (clipped_values - p5) / (p95 - p5)
        else:  # cost
            # Lower values are better
            standardized = (p95 - clipped_values) / (p95 - p5)
        
        # Ensure values are between 0 and 1
        Data[std_col_name] = standardized.clip(0, 1)
    
    print(f"Standardized {actual_field} -> {std_col_name} ({direction})")

Standardized NearbySchools -> STD_NearbySchools (benefit)
Standardized NearbyRestuarants -> STD_NearbyRestuaran (benefit)
Standardized NearbyGrocery -> STD_NearbyGrocery (benefit)
Standardized WalkabilityIndex -> STD_WalkabilityInde (benefit)
Standardized NearbyParks -> STD_NearbyParks (benefit)
Standardized Nearbyhealthcare -> STD_Nearbyhealthcar (benefit)
Standardized DistanceToHighway -> STD_DistanceToHighw (cost)
Standardized DistanceToRegionalRail -> STD_DistanceToRegio (cost)
Standardized DistanceToAirport -> STD_DistanceToAirpo (cost)
Standardized 2024_Total_Crime_Index -> STD_2024_Total_Crim (cost)
Standardized NewConstructionActivity -> STD_NewConstruction (benefit)
Standardized NearbyCommercialDevelopment -> STD_NearbyCommercia (benefit)
Standardized 2025_NAICS_44_45_722_Expenditures -> STD_2025_NAICS_44_4 (benefit)
Standardized NearbyResidentialDevelopment -> STD_NearbyResidenti (benefit)
Standardized 2025_Unemployment_Rate -> STD_2025_Unemployme (cost)
Standardized 2030_Tot

#### Create Group Scoring

In [13]:
# --- 5. Calculate weighted group scores ---
print("\nCalculating weighted group scores...")

# Group criteria by category with their weights
criteria_by_group = {}
for fld, group, weight, direction in criteria:
    if group not in criteria_by_group:
        criteria_by_group[group] = []
    criteria_by_group[group].append((fld, weight))

# Calculate weighted group scores (not simple means)
for group, group_criteria in criteria_by_group.items():
    group_score_col = f"{group}_Score"
    Data[group_score_col] = 0.0
    total_group_weight = 0.0
    
    # Add weighted contributions from each criterion
    for fld, weight in group_criteria:
        std_col_name = f"STD_{fld.replace(' ', '_').replace('(', '').replace(')', '')[:15]}"
        if std_col_name in Data.columns:
            Data[group_score_col] += Data[std_col_name].fillna(0) * weight
            total_group_weight += weight
    
    # Normalize by total weight if needed (in case weights don't sum to 1.0)
    if total_group_weight > 0 and abs(total_group_weight - 1.0) > 0.01:
        Data[group_score_col] = Data[group_score_col] / total_group_weight
        print(f"Group {group}: {len(group_criteria)} weighted criteria -> {group_score_col} (normalized by {total_group_weight:.3f})")
    else:
        print(f"Group {group}: {len(group_criteria)} weighted criteria -> {group_score_col}")


Calculating weighted group scores...
Group Access: 10 weighted criteria -> Access_Score
Group Market: 10 weighted criteria -> Market_Score
Group Infra: 3 weighted criteria -> Infra_Score
Group Reg: 2 weighted criteria -> Reg_Score


In [14]:
# --- 6. Calculate final composite score using group weights ---
print("\nCalculating final composite score...")
group_weights = {"Access": 0.30, "Market": 0.40, "Infra": 0.20, "Reg": 0.10}
Data['FinalScore'] = 0.0

for group, weight in group_weights.items():
    score_col = f"{group}_Score"
    if score_col in Data.columns:
        Data['FinalScore'] += Data[score_col].fillna(0) * weight
        print(f"Added {group}_Score with weight {weight}")

# --- 7. Calculate rankings ---
print("\nCalculating rankings...")
Data['ScoreRanking'] = Data['FinalScore'].rank(method='dense', ascending=False).astype(int)


Calculating final composite score...
Added Access_Score with weight 0.3
Added Market_Score with weight 0.4
Added Infra_Score with weight 0.2
Added Reg_Score with weight 0.1

Calculating rankings...


In [15]:
original_fields = []
for fld, group, weight, direction in criteria:
    if fld in field_mapping:
        original_fields.append(field_mapping[fld])

# Add sub-scores and final score
sub_score_columns = ['ScoreRanking','Ranking','Access_Score', 'Market_Score', 'Infra_Score', 'Reg_Score']
output_columns = original_fields + sub_score_columns + ['FinalScore']

# Filter to only existing columns
existing_output_columns = [col for col in output_columns if col in Data.columns]

# Save filtered dataset
output_file = 'ArlingtonMarketReadiness.xlsx'
Data[existing_output_columns].to_excel(output_file, index=False)
print(f"\nResults saved to: {output_file}")
print(f"Output contains {len(existing_output_columns)} columns:")
print(f"  - {len(original_fields)} criteria fields")
print(f"  - 4 sub-score columns (Access_Score, Market_Score, Infra_Score, Reg_Score)")
print(f"  - 1 final score column")

print("\nColumns included:")
print(f"Criteria: {[col for col in existing_output_columns if col in original_fields]}")
print(f"Sub-scores: {[col for col in existing_output_columns if col in sub_score_columns]}")
print(f"Final: FinalScore")

print("\nAnalysis complete!")

# Display first 10 rows with scores for verification
print("\nFirst 10 rows with scores:")
score_display_cols = ['Access_Score', 'Market_Score', 'Infra_Score', 'Reg_Score', 'FinalScore']
available_score_cols = [col for col in score_display_cols if col in Data.columns]
print(Data[available_score_cols].head(10).to_string(index=True))


Results saved to: ArlingtonMarketReadiness.xlsx
Output contains 32 columns:
  - 25 criteria fields
  - 4 sub-score columns (Access_Score, Market_Score, Infra_Score, Reg_Score)
  - 1 final score column

Columns included:
Criteria: ['NearbySchools', 'NearbyRestuarants', 'NearbyGrocery', 'WalkabilityIndex', 'NearbyParks', 'Nearbyhealthcare', 'DistanceToHighway', 'DistanceToRegionalRail', 'DistanceToAirport', '2024_Total_Crime_Index', 'NewConstructionActivity', 'NearbyCommercialDevelopment', '2025_NAICS_44_45_722_Expenditures', 'NearbyResidentialDevelopment', '2025_Unemployment_Rate', '2030_Total_Population', '2025_Median_Home_Value', '2025_Total_Businesses__NAICS_', '2025_Vacant_Housing_Units', '2025_Median_Monthly_Contract_Rent', 'StreetCondition', 'Avg5yrTraffic', 'NearbyPolice', 'TotalZoningAcres', 'NearbyBrownfields']
Sub-scores: ['ScoreRanking', 'Ranking', 'Access_Score', 'Market_Score', 'Infra_Score', 'Reg_Score']
Final: FinalScore

Analysis complete!

First 10 rows with scores:
  

In [16]:
arcpy.conversion.ExportTable("ArlingtonMarketReadiness.xlsx","ArlingtonMarketReadinessFinalScore","",
                             "NOT_USE_ALIAS","",None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessFinalScore'>

In [19]:
excel_sheet = r"ArlingtonMarketReadiness.xlsx\Sheet1$"

# Export Excel sheet into a table
arcpy.conversion.ExportTable(excel_sheet, "ArlingtonMarketReadinessFinalScore", "", "NOT_USE_ALIAS","", None)

# Create field mapping
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessPG")
fieldmapping.addTable("ArlingtonMarketReadinessFinalScore")

# Join PG to exported table
Assessment = arcpy.management.AddJoin("ArlingtonMarketReadinessPG","Ranking","ArlingtonMarketReadinessFinalScore",
                                      "Ranking", "KEEP_ALL","NO_INDEX_JOIN_FIELDS","NO_REBUILD_INDEX")

# Export joined features
arcpy.conversion.ExportFeatures( Assessment, "ArlingtonMarketReadinessScore", "", "NOT_USE_ALIAS", fieldmapping,
    None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessScore'>

In [21]:
table_fields = [f.name for f in arcpy.ListFields('ArlingtonMarketReadinessScore') if f.name !='Ranking']
table_fields

['OBJECTID',
 'Shape',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Location',
 'Override',
 'NearbySchools',
 'NearbyRestuarants',
 'NearbyGrocery',
 'WalkabilityIndex',
 'NearbyParks',
 'Nearbyhealthcare',
 'DistanceToHighway',
 'DistanceToRegionalRail',
 'DistanceToAirport',
 'F2024_Total_Crime_Index',
 'NewConstructionActivity',
 'NearbyCommercialDevelopment',
 'F2025_NAICS_44_45_722_Expenditu',
 'NearbyResidentialDevelopment',
 'F2025_Unemployment_Rate',
 'F2030_Total_Population',
 'F2025_Median_Home_Value',
 'F2025_Total_Businesses__NAICS_',
 'F2025_Vacant_Housing_Units',
 'F2025_Median_Monthly_Contract_R',
 'StreetCondition',
 'Avg5yrTraffic',
 'NearbyPolice',
 'TotalZoningAcres',
 'NearbyBrownfields',
 'ScoreRanking',
 'Access_Score',
 'Market_Score',
 'Infra_Score',
 'Reg_Score',
 'FinalScore',
 'Shape_Length',
 'Shape_Area']

In [27]:
# Inputs
fc = "ArlingtonMarketAssessmentAreas2025" # polygons
tbl = "ArlingtonMarketReadinessFinalScore"        # table with 8 records
join_field = "Ranking"                        # common key

# --- Step 1: Get list of fields to join ---
# exclude OID, shape, and the join field
exclude = [f.name for f in arcpy.ListFields(tbl) if f.type in ("OID", "Geometry")]
table_fields = [f.name for f in arcpy.ListFields(tbl) if f.name not in exclude + [join_field]]

# --- Step 2: Build lookup dictionary ---
lookup = {}
with arcpy.da.SearchCursor(tbl, [join_field] + table_fields) as cursor:
    for row in cursor:
        key = row[0]
        lookup[key] = row[1:]  # everything else

# --- Step 3: Make sure fields exist in the feature class ---
fc_fields = [f.name for f in arcpy.ListFields(fc)]
for fld in table_fields:
    if fld not in fc_fields:
        # you can customize type/length — here I match the source table
        fld_def = [f for f in arcpy.ListFields(tbl) if f.name == fld][0]
        arcpy.AddField_management(fc, fld, fld_def.type, fld_def.precision,
                                  fld_def.scale, fld_def.length)

# --- Step 4: Update feature class ---
with arcpy.da.UpdateCursor(fc, [join_field] + table_fields) as cursor:
    for row in cursor:
        key = row[0]
        if key in lookup:
            row[1:] = lookup[key]
            cursor.updateRow(row)

In [24]:
fc_fields

['OBJECTID',
 'Shape',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Ranking',
 'Location',
 'Value',
 'Override',
 'Shape_Length',
 'Shape_Area']